### Short Term Memory

In [ ]:
!pip install langchain langchain_community langchain_groq langgraph

In [ ]:
#Importing required libraries
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List, Annotated
from langgraph.checkpoint.memory import InMemorySaver
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph.message import AnyMessage, add_messages
import os

In [ ]:
#Loading API key
from google.colab import userdata
groq_api_key = userdata.get("GROQ_API_KEY")

In [ ]:
#LLM
llm = ChatGroq(model = "llama-3.3-70b-versatile", api_key = groq_api_key)

In [ ]:
#Define the State
class State(TypedDict):
  messages: Annotated[List[AnyMessage], add_messages]

In [ ]:
#Prompt
prompts = ChatPromptTemplate.from_messages(
    [
        ("system", "{system_messages}"),
        MessagesPlaceholder("messages")
    ]
)

llm_model = prompts | llm

#Build the Graph
workflow = StateGraph(State)

def chatnode(state:State):
  system_messages = "You are an assitant"
  state["messages"] = llm_model.invoke({"system_messages": system_messages, "messages": state["messages"]})
  return state

#Add node
workflow.add_node("chatnode", chatnode)
#Add edges
workflow.add_edge(START, "chatnode")
workflow.add_edge("chatnode", END)

#Compile the graph
graph = workflow.compile()

In [ ]:
#First question
response = graph.invoke({"messages": "Heyy, My name is Krish"})
for message in response["messages"]:
  message.pretty_print()

================================ Human Message =================================

Heyy, My name is Krish
================================== Ai Message ==================================

Hello Krish! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm all ears!


In [ ]:
#Second question
response2 = graph.invoke({"messages": "Heyy, Do you remember my name?"})
for message in response2["messages"]:
  message.pretty_print()

================================ Human Message =================================

Heyy, Do you remember my name?
================================== Ai Message ==================================

I'm happy to chat with you. However, I'm a large language model, I don't have personal memories, so I don't recall previous conversations or names. Each time you interact with me, it's a new conversation. If you'd like to share your name, I'd be happy to use it in our conversation though!


In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

#Build the Graph
workflow = StateGraph(State)

def chatnode(state:State):
  system_messages = "You are an assitant"
  state["messages"] = llm_model.invoke({"system_messages": system_messages, "messages": state["messages"]})
  return state

#Add node
workflow.add_node("chatnode", chatnode)
#Add edges
workflow.add_edge(START, "chatnode")
workflow.add_edge("chatnode", END)

checkpointer = InMemorySaver()

#Compile the graph
graph = workflow.compile(checkpointer = checkpointer)

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}

#First question
response = graph.invoke({"messages": "Heyy, My name is Krish"}, config = config1)
for message in response["messages"]:
  message.pretty_print()

================================ Human Message =================================

Heyy, My name is Krish
================================== Ai Message ==================================

Hello Krish! It's nice to meet you! Is there something I can help you with or would you like to chat? I'm all ears!


In [ ]:
#Second question
response2 = graph.invoke({"messages": "Heyy, Do you remember my name?"}, config = config1)
for message in response2["messages"]:
  message.pretty_print()

================================ Human Message =================================

Heyy, My name is Krish
================================== Ai Message ==================================

Hello Krish! It's nice to meet you! Is there something I can help you with or would you like to chat? I'm all ears!
================================ Human Message =================================

Heyy, Do you remember my name?
================================== Ai Message ==================================

Your name is Krish, right? I remember! We just met a minute ago. How can I assist you today, Krish?


In [ ]:
graph.get_state(config= config1)

StateSnapshot(values={'messages': [HumanMessage(content='Heyy, My name is Krish', additional_kwargs={}, response_metadata={}, id='ec38309f-ac25-45bb-814d-0ab77356bd9c'), AIMessage(content="Hello Krish! It's nice to meet you! Is there something I can help you with or would you like to chat? I'm all ears!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 47, 'total_tokens': 78, 'completion_time': 0.065060804, 'prompt_time': 0.016321655, 'queue_time': 0.220762273, 'total_time': 0.081382459}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b51da721-38d5-4e72-9b50-b36a933b6d70-0', usage_metadata={'input_tokens': 47, 'output_tokens': 31, 'total_tokens': 78}), HumanMessage(content='Heyy, Do you remember my name?', additional_kwargs={}, response_metadata={}, id='fa102a97-7dc4-48a3-9491-e4c4e01db61b'), AIMessage(content='Your name i

In [ ]:
list(graph.get_state_history(config1))

[StateSnapshot(values={'messages': [HumanMessage(content='Heyy, My name is Krish', additional_kwargs={}, response_metadata={}, id='ec38309f-ac25-45bb-814d-0ab77356bd9c'), AIMessage(content="Hello Krish! It's nice to meet you! Is there something I can help you with or would you like to chat? I'm all ears!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 47, 'total_tokens': 78, 'completion_time': 0.065060804, 'prompt_time': 0.016321655, 'queue_time': 0.220762273, 'total_time': 0.081382459}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b51da721-38d5-4e72-9b50-b36a933b6d70-0', usage_metadata={'input_tokens': 47, 'output_tokens': 31, 'total_tokens': 78}), HumanMessage(content='Heyy, Do you remember my name?', additional_kwargs={}, response_metadata={}, id='fa102a97-7dc4-48a3-9491-e4c4e01db61b'), AIMessage(content='Your name 

### Long-Term Memory
 - Use long-term memory to store user-specific or application-specific data across conversations

In [ ]:
from langgraph.store.memory import InMemoryStore

#Build the Graph
workflow = StateGraph(State)

def chatnode(state:State):
  system_messages = "You are an assitant"
  state["messages"] = llm_model.invoke({"system_messages": system_messages, "messages": state["messages"]})
  return state

#Add node
workflow.add_node("chatnode", chatnode)
#Add edges
workflow.add_edge(START, "chatnode")
workflow.add_edge("chatnode", END)

store = InMemoryStore()

#Compile the graph
graph = workflow.compile(store = store)


In [ ]:
config1 = {"configurable": {"thread_id": "2"}}

#First question
response = graph.invoke({"messages": "Heyy, My name is Krish and I like Vadapav"}, config = config1)
for message in response["messages"]:
  message.pretty_print()

================================ Human Message =================================

Heyy, My name is Krish and I like Vadapav
================================== Ai Message ==================================

Namaste Krish, nice to meet you. Vadapav is a popular Indian street food, isn't it? It's a staple in Mumbai, with a crispy vada (fried potato dumpling) served in a soft pav (bread bun) with a variety of chutneys. What's your favorite way to enjoy Vadapav - with a hot cup of chai or as a quick snack on-the-go?


In [ ]:
config1 = {"configurable": {"thread_id": "2"}}

#First question
response = graph.invoke({"messages": "Heyy, Do you not what I like?"}, config = config1)
for message in response["messages"]:
  message.pretty_print()

================================ Human Message =================================

Heyy, Do you not what I like?
================================== Ai Message ==================================

I'm happy to chat with you. Since we just started talking, I don't have any information about your preferences or interests yet. I'm a blank slate, ready to learn and help. Would you like to share what you're into, or is there something specific you'd like to talk about? I'm all ears!
